# Word Sense Disambiguation 

In [16]:
txt_file = open("disambdata.txt", "r")
data = txt_file.readlines()
corpus = []
sense = []
for i in data:
    corpus.append(i.split(" : ")[0])
    sense.append(i.split(" : ")[1].split("\n")[0])

In [17]:
import string
import nltk ,re
import math
import numpy as np
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
stop_words = set(stopwords.words('english'))
for i in range(0, len(corpus)):
    corpus[i] = nltk.word_tokenize(corpus[i])
corpus

[['He', 'acts', 'as', 'the', 'IT', 'department', 'chair'],
 ['He', 'is', 'seated', 'in', 'a', 'steel', 'chair'],
 ['chair',
  'does',
  'not',
  'interfere',
  'in',
  'day',
  'to',
  'day',
  'company',
  'matters'],
 ['I', 'need', 'a', 'chair', 'to', 'sit'],
 ['He', 'is', 'lying', 'down', 'on', 'a', 'chair']]

In [19]:
ws= []
filtered_corpus = []
for i in corpus:
    stop = []
    for j in i:
        if j not in stop_words and j != "'s":
            ws.append(j)
        else:
            stop.append(j)
    filtered_corpus.append(i)
    for k in stop:
        filtered_corpus[-1].remove(k)
ws

['He',
 'acts',
 'IT',
 'department',
 'chair',
 'He',
 'seated',
 'steel',
 'chair',
 'chair',
 'interfere',
 'day',
 'day',
 'company',
 'matters',
 'I',
 'need',
 'chair',
 'sit',
 'He',
 'lying',
 'chair']

In [20]:
vocab = []
for i in ws:
    if i not in vocab:
        vocab.append(i)
vocab

['He',
 'acts',
 'IT',
 'department',
 'chair',
 'seated',
 'steel',
 'interfere',
 'day',
 'company',
 'matters',
 'I',
 'need',
 'sit',
 'lying']

In [21]:
def count(i, j):
    counter = 0
    for k in i:
        if k==j:
            counter+=1
    return counter

In [22]:
sense_data = {}
for i in sense:
    if i not in sense_data:
        sense_data[i] = count(sense, i)
sense_data

{'Position': 2, 'furniture': 3}

In [25]:
target_sentence = "I'm the department chair of IT"
target_token = nltk.word_tokenize(target_sentence)
target_ws = []
for i in target_token:
    if i not in stop_words and i != "'m":
        target_ws.append(i)
target_ws

['I', 'department', 'chair', 'IT']

In [27]:
class_count = {}
for i in target_ws:
    for k in sense_data.keys():
        class_count[i+k] = 0
    for j in range(0, len(filtered_corpus)):
        word_count = count(filtered_corpus[j], i)
        class_count[i+sense[j]]+=word_count
class_count

{'IPosition': 0,
 'Ifurniture': 1,
 'departmentPosition': 1,
 'departmentfurniture': 0,
 'chairPosition': 2,
 'chairfurniture': 3,
 'ITPosition': 1,
 'ITfurniture': 0}

In [28]:
result = {}
for i in sense_data.keys():
    prob = sense_data[i]/len(filtered_corpus)
    for j in target_ws:
        prob *= (class_count[j+i]+1)/(sense_data[i]+len(vocab))
    result[i] = prob

In [29]:
result

{'Position': 5.747057626225739e-05, 'furniture': 4.572473708276177e-05}

In [30]:
max(result, key=result.get)

'Position'

In [31]:
from math import log
log_result = {}
for i in sense_data.keys():
    prob = log(2, sense_data[i]/len(filtered_corpus))
    for j in target_ws:
        prob *= log(2,(class_count[j+i]+1)/(sense_data[i]+len(vocab)))
    log_result[i] = prob

In [32]:
log_result

{'Position': -0.007758195614353548, 'furniture': -0.011344950051796298}

In [33]:
max(log_result, key=result.get)

'Position'